In [ ]:
import riiideducation
import dask.dataframe as dd

env = riiideducation.make_env()

In [ ]:
content_acc = dd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                usecols=[3,7], dtype={'content_id': 'int16','answered_correctly':'int8'}
              ).query('answered_correctly != -1').groupby('content_id')['answered_correctly'].mean().compute().to_dict()

In [ ]:
def add_content_acc(x):
    if x in content_acc.keys():
        return content_acc[x]
    else:
        return 0.5

iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = test_df['content_id'].apply(add_content_acc).values
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])